In [46]:
import numpy as np
import pickle
import itertools
import pandas as pd

In [28]:
Nt, Ng, Nw, Nd, Nh = 5, 2, 2, 6, 8
Ntg = Nt*Ng + 1
T = Nw*Nd*Nh
Nx = T*Ntg

def v(tg, w, d, h):
    return T*tg + (w * Nd + d)*Nh + h

N6 = 7
N4 = 4

In [48]:
Q = np.zeros((Nx+N6+N4, Nx+N6+N4))
offset = 0

# no simultaneous lessons in one group
P_LiG = 1
for (w,d,h, g) in itertools.product(range(Nw), range(Nd), range(Nh), range(Ng)):
    # sum(slots[n] for n in range(5)) <= 1  <=>  sum(slots[n1]*slots[n2] for perms(n1, n2)) 
    for (n1, n2) in itertools.permutations(range(Nt), 2):
        Q[v(1+g+2*n1,w,d,h),v(1+g+2*n2,w,d,h)] += P_LiG * 0.5

# no simultaneuos lessons with one teacher
P_LoT = 1
for (w,d,h, n) in itertools.product(range(Nw), range(Nd), range(Nh), range(Nt)):
    # sum(slots[g] for n in range(2)) <= 1  <=>  sum(slots[g1]*slots[g2] for perms(g1, g2)) 
    for (g1, g2) in itertools.permutations(range(Ng), 2):
        Q[v(1+g1+2*n,w,d,h),v(1+g2+2*n,w,d,h)] += P_LoT * 0.5
        
# no more that 6 lecs a day
P_nm6 = 1
for i in range(N6):
    Q[Nx+i, Nx+i] += P_nm6
for g in range(Ng):
    for (w, d) in itertools.product(range(Nw), range(Nd)):
        xinds = [v(1+2*n+g,w,d,h) for n in range(Nt) for h in range(Nh)]
        inds = [*xinds, *list(range(N6))]
        mults = np.concatenate((np.ones((Nt*Nh,)), np.arange(N6)))
        for i1,m1 in zip(inds, mults):
            for i2,m2 in zip(inds, mults):
                Q[i1, i2] += P_nm6 * m1*m2
        
# less than 4 lectures on each teacher
P_nm4 = 1
for i in range(N4):
    Q[Nx+N6+i, Nx+N6+i] += P_nm4
for g in range(Ng):
    for (w, d, n) in itertools.product(range(Nw), range(Nd), range(Nt)):
        xinds = [v(1+2*n+g,w,d,h) for h in range(Nh)]
        inds = [*xinds, *list(range(N6, N6+N4))]
        mults = np.concatenate((np.ones((Nt*Nh,)), np.arange(N4)))
        for i1,m1 in zip(inds, mults):
            for i2,m2 in zip(inds, mults):
                Q[i1, i2] += P_nm4 * m1*m2
        
# needed 8 lectures with one lector
P_L8 = 0
for (n,g) in itertools.product(range(Nt), range(Ng)):
    xinds = [v(1+2*n+g,w,d,h) for (w,d,h) in itertools.product(range(Nw), range(Nd), range(Nh))]
    offset += 64
    for i1 in xinds:
        Q[i1, i1] -= 16
        for i2 in xinds:
            Q[i1, i2] += P_L8

# wishes of teachers
# TODO - may be make it quadratic
P_TW = 1
cants = [2, 0, 5, 1, 1]
for n, d in zip(range(Nt), cants):
    for (g, w, h) in itertools.product(range(Ng), range(Nw), range(Nh)):
        Q[v(1+g+2*n,w,d,h),v(1+g+2*n,w,d,h)] += P_TW

In [49]:
import neal

In [50]:
variables = {i:i for i in range(Nx+N6+N4)}
linear = {i:Q[i,i] for i in range(Nx+N6+N4)}
quadratic = {(i,j):Q[i,j]+Q[j,i] for j in range(Nx+N6+N4) for i in range(j)}

In [51]:
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=10)
print(sampleset.first.energy)

-1228.0


In [ ]:
pd.DataFrame(columns=("Group", "Week", "Day", "Hour", "Subject", "Name"))
custom_dict_group = {'QC_1': 0, 'QC_2': 1}
custom_dict_week = {'неделя 1': 0, 'неделя 2': 1}
custom_dict_day = {'пн': 0, 'вт': 1, 'ср': 2, 'чт': 3, 'пт': 4, 'сб': 5, 'вс': 6}
custom_dict_hour = {'9:00-10:00': 0, '10:00-11:00': 1, '11:00-12:00': 2, '12:00-13:00': 3, '13:00-14:00': 4, '14:00-15:00': 5, '15:00-16:00': 6, '16:00-17:00': 7}
custom_dict_subject = {'Квантовая механика': 0, 'Квантовая теория информации': 1, 'Квантовые вычисления': 2, 'Сложность квантовых алгоритмов': 3, 'Квантовые алгоритмы в логистике': 4,
                        'Квантовое машинное обучение': 5, 'Моделирование квантовых систем': 6, 'Квантовые алгоритмы в химии': 7,'Физическая реализация квантовых компьютеров': 8,
                        'Моделирование квантовых алгоритмов': 9, 'нет занятий': 10}

custom_dict_teacher = {'Иванов': 0, 'Петров': 1, 'Сидоров': 2, 'Карпов': 3, 'Соколов': 4, 'нет занятий': 10}
custom_dict_teacheryn = {'Иванов': 1, 'Петров': 1, 'Сидоров': 1, 'Карпов': 1, 'Соколов': 1, 'нет занятий': 0}


In [45]:
sampleset.first

Sample(sample={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 0, 16: 1, 17: 0, 18: 0, 19: 1, 20: 0, 21: 1, 22: 0, 23: 1, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 0, 31: 1, 32: 1, 33: 0, 34: 1, 35: 0, 36: 1, 37: 0, 38: 0, 39: 1, 40: 1, 41: 0, 42: 1, 43: 1, 44: 1, 45: 0, 46: 1, 47: 1, 48: 0, 49: 0, 50: 0, 51: 1, 52: 0, 53: 0, 54: 1, 55: 0, 56: 1, 57: 1, 58: 1, 59: 0, 60: 1, 61: 0, 62: 0, 63: 1, 64: 1, 65: 1, 66: 0, 67: 0, 68: 0, 69: 1, 70: 0, 71: 0, 72: 1, 73: 1, 74: 1, 75: 1, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 1, 82: 1, 83: 0, 84: 0, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 0, 92: 0, 93: 1, 94: 1, 95: 1, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 1, 103: 0, 104: 1, 105: 0, 106: 1, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 1, 116: 0, 117: 1, 118: 0, 119: 0, 120: 0, 121: 0, 122: 1, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 1, 132: 0, 133: 0, 134: 0, 135: 1, 136: 0,

In [36]:
variables

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

In [1]:
import pyqubo as pq
import itertools
import neal

In [2]:
Nt, Ng, Np, Ns, T = 5, 2, 2, 4, 7
Nw, Nd, Nh = 2, 6, 8
nw, nd, nh = 1, 3, 3

In [3]:
def delta(x1, x2, n):
    answ = 1
    if isinstance(x2, int):
        for i in range(n):
            if (x2 >> i) % 2:
                answ *= x1[i]
            else:
                answ *= 1 - x1[i]
    else:
        for i in range(n):
            answ *= (1 - x1[i] - x2[i] + 2*x1[i]*x2[i])
    return answ

def make_inequality_constraint(expr, maximum, name, strength):
    cvar = pq.OneHotEncInteger(name+"_var", value_range=(1,maximum), strength=strength)
    return pq.Constraint((expr - cvar)**2, name)
    

In [4]:
Nt, Ng, Nw, Nd, Nh = 5, 2, 2, 6, 8
Ntg = Nt*Ng + 1
slots = pq.Array.create("slots", shape=(Ntg, Nw, Nd, Nh), vartype='BINARY') # tutor, group, subject, slot, week, day, hour

In [5]:
# no simultaneous lessons in one group
H_time_c = 0
for (w,d,h, g) in itertools.product(range(Nw), range(Nd), range(Nh), range(Ng)):
    # sum(slots[n] for n in range(5)) <= 1  <=>  sum(slots[n1]*slots[n2] for perms(n1, n2)) 
    for (n1, n2) in itertools.permutations(range(Nt), 2):
        H_time_c += 0.5*slots[1+g+2*n1,w,d,h]*slots[1+g+2*n2,w,d,h]

# no simultaneuos lessons with one teacher
H_teach_c = 0
for (w,d,h, n) in itertools.product(range(Nw), range(Nd), range(Nh), range(Nt)):
    # sum(slots[g] for n in range(2)) <= 1  <=>  sum(slots[g1]*slots[g2] for perms(g1, g2)) 
    for (g1, g2) in itertools.permutations(range(Ng), 2):
        H_time_c += 0.5*slots[1+g1+2*n,w,d,h]*slots[1+g2+2*n,w,d,h]
        
# no more that 6 lecs a day
H_6_lecs_c = 0
str_6_lecs_c = pq.Placeholder("str6lecs_c")
for g in range(Ng):
    for (w, d) in itertools.product(range(Nw), range(Nd)):
        H_6_lecs_c += make_inequality_constraint(sum([slots[1+2*n+g,w,d,h] for n in range(Nt) for h in range(Nh)]), 6, "6_lecs_c", strength=str_6_lecs_c)
    
# less than 4 lectures on each teacher
H_4_lecs_c = 0
str_4_lecs_c = pq.Placeholder("str4lecs_c")
for g in range(Ng):
    for (w, d, n) in itertools.product(range(Nw), range(Nd), range(Nt)):
        H_4_lecs_c += make_inequality_constraint(sum([slots[1+2*n+g,w,d,h] for h in range(Nh)]), 4, "4_lecs_c", strength=str_4_lecs_c)

# needed 8 lectures with one lector
H_8_lecs_c = 0
for (n,g) in itertools.product(range(Nt), range(Ng)):
    H_8_lecs_c += pq.Constraint((sum([slots[1+2*n+g,w,d,h] for (w,d,h) in itertools.product(range(Nw), range(Nd), range(Nh))]) - 8)**2, "8_lecs_c")



In [6]:
H = H_time_c + H_teach_c + H_4_lecs_c + H_6_lecs_c + H_8_lecs_c
model = H.compile()

In [ ]:
feed_dict={"str4lecs_c":1,"str6lecs_c":1}
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [7]:
model.variables

['slots[10][1][5][7]',
 'slots[10][1][5][6]',
 'slots[10][1][5][5]',
 'slots[10][1][5][4]',
 'slots[10][1][5][3]',
 'slots[10][1][5][2]',
 'slots[10][1][5][1]',
 'slots[10][1][5][0]',
 'slots[10][1][4][7]',
 'slots[10][1][4][6]',
 'slots[10][1][4][5]',
 'slots[10][1][4][4]',
 'slots[10][1][4][3]',
 'slots[10][1][4][2]',
 'slots[10][1][4][1]',
 'slots[10][1][4][0]',
 'slots[10][1][3][7]',
 'slots[10][1][3][6]',
 'slots[10][1][3][5]',
 'slots[10][1][3][4]',
 'slots[10][1][3][3]',
 'slots[10][1][3][2]',
 'slots[10][1][3][1]',
 'slots[10][1][3][0]',
 'slots[10][1][2][7]',
 'slots[10][1][2][6]',
 'slots[10][1][2][5]',
 'slots[10][1][2][4]',
 'slots[10][1][2][3]',
 'slots[10][1][2][2]',
 'slots[10][1][2][1]',
 'slots[10][1][2][0]',
 'slots[10][1][1][7]',
 'slots[10][1][1][6]',
 'slots[10][1][1][5]',
 'slots[10][1][1][4]',
 'slots[10][1][1][3]',
 'slots[10][1][1][2]',
 'slots[10][1][1][1]',
 'slots[10][1][1][0]',
 'slots[10][1][0][7]',
 'slots[10][1][0][6]',
 'slots[10][1][0][5]',
 'slots[10]

In [ ]:
feed_dict={"str4lecs_c":1,"str6lecs_c":1}
qubo, offset = model.to_qubo(feed_dict=feed_dict)
bqm = model.to_bqm(feed_dict=feed_dict)
bqm.normalize()
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=10, sweeps=1000, beta_range=(1.0, 50.0))
dec_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
best = min(dec_samples, key=lambda x: x.energy)

In [32]:
len(best.sample)

960

In [35]:
best.sample

{'slots[2][0][3][0]': 0,
 'slots[10][1][0][5]': 0,
 'slots[9][1][1][6]': 0,
 'slots[10][0][5][7]': 0,
 'slots[9][1][2][7]': 0,
 'slots[10][1][4][1]': 0,
 'slots[2][0][5][6]': 0,
 'slots[6][1][1][7]': 0,
 'slots[5][0][5][7]': 0,
 'slots[10][0][0][0]': 0,
 'slots[2][0][3][1]': 0,
 'slots[10][1][0][4]': 0,
 'slots[9][1][1][7]': 0,
 'slots[10][0][5][6]': 0,
 'slots[9][1][2][6]': 0,
 'slots[10][1][4][0]': 1,
 'slots[2][0][5][7]': 0,
 'slots[6][1][1][6]': 0,
 'slots[5][0][5][6]': 1,
 'slots[10][0][0][1]': 0,
 'slots[2][0][3][2]': 0,
 'slots[10][1][0][7]': 1,
 'slots[9][1][1][4]': 0,
 'slots[10][0][5][5]': 0,
 'slots[9][1][2][5]': 0,
 'slots[10][1][4][3]': 0,
 'slots[2][0][5][4]': 0,
 'slots[6][1][1][5]': 0,
 'slots[5][0][5][5]': 0,
 'slots[10][0][0][2]': 0,
 'slots[2][0][3][3]': 0,
 'slots[10][1][0][6]': 0,
 'slots[9][1][1][5]': 0,
 'slots[10][0][5][4]': 1,
 'slots[9][1][2][4]': 1,
 'slots[10][1][4][2]': 0,
 'slots[2][0][5][5]': 0,
 'slots[6][1][1][4]': 0,
 'slots[5][0][5][4]': 1,
 'slots[10

In [33]:
Ntg*Nw*Nd*Nh

1056

In [34]:
lst = [best.sample[f'slots[{ng}][{w}][{d}][{h}]'] for (ng,w,d,h) in itertools.product(range(Ntg), range(Nw), range(Nd), range(Nh))]
print(f"selection = {lst}")
print(f"sum of the values = {-best.energy}")

KeyError: 'slots[0][0][0][0]'

In [4]:
slots = pq.Array.create("slots", shape=(Nt, Ng, Np, Ns, T), vartype='BINARY') # tutor, group, subject, slot, time in binary enc

In [6]:
# No group allowed to have 2 lessons at one time 
H_group_c = [0,0]
for g in range(Ng):
    for (n1, p1, s1, n2, p2, s2) in itertools.product(range(Nt), range(Np), range(Ns), repeat=2):
        if n1 == n2 and p1 == p2 and s1 == s2:
            continue
        else:
            for b in range(T):
                H_group_c[g] += -(slots[n1, g, p1, s1, b] - slots[n2, g, p2, s2, b])**2

# No tutor allowed to give 2 lessons at one time
H_tutor_c = [0,0,0,0,0]
for n in range(Nt):
    for (p1, g1, s1, p2, g2, s2) in itertools.product(range(Ng), range(Np), range(Ns), repeat=2):
        if p1 == p2 and g1 == g2 and s1 == s2:
            continue
        else:
            for b in range(T):
                H_tutor_c[n] += -(slots[n, g1, p1, s1, b] - slots[n, g2, p2, s2, b])**2

# No more than 6 lessons in one day for a group
H_lessonatday_c = 0
str_lessonatday = pq.Placeholder("str_lessonatday")
for (g, w, d) in itertools.product(range(Ng), range(Nw), range(Nd)):
    H_lessonatday_c += make_inequality_constraint(sum([delta(slots[n,g,p,s,3:], (1 << nd)*w+d, 4) for n in range(Nt) for p in range(Np) for s in range(Ns)]), 
                                                             6, f"no_more_6less_c_g{g}_w{w}_d{d}", strength=str_lessonatday)
    
# No more than 2 lessons on 1 subject in a day
H_subjectsatday_c = 0
str_subjectsatday = pq.Placeholder("str_subjectsatday")
for (g, n, p, w, d) in itertools.product(range(Ng), range(Nt), range(Np), range(Nw), range(Nd)):
    H_subjectsatday_c += make_inequality_constraint(sum([delta(slots[n,g,p,s,3:], (1 << nd)*w+d, 4) for s in range(Ns)]), 
                                                             2, f"no_more_2less_c_g{g}_n{n}_p{p}_w{w}_d{d}", strength=str_subjectsatday)

# Tutors wishes
#H_tutorwishes_c = 0
#cants = [2, 0, 5, 1, 1]
#for n in Nt:
#    for (g, p, s, w) in itertools.product(range(Ng), range(Np), range(Ns), range(Nw)):
#        H_tutorwishes_c += -sum([(slots[]) for dbit in range(nd)])


In [2]:
6*8*2 * 11*2

2112

In [ ]:
#+ H_lessonatday_c + H_subjectsatday_c
%time
H = sum(H_group_c) + sum(H_tutor_c) + H_lessonatday_c
model = H.compile()

In [34]:
feed_dict={}
qubo, offset = model.to_qubo(feed_dict=feed_dict)
bqm = model.to_bqm(feed_dict=feed_dict)
bqm.normalize()
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=10, sweeps=1000, beta_range=(1.0, 50.0))
dec_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
best = min(dec_samples, key=lambda x: x.energy)

In [35]:
best.constraints()

{}

In [36]:
lst = [[sum([best.sample[f'slots[{n}][{g}][{p}][{s}][{b}]'] * 2**b for b in range(T)]) for n in range(Nt)] for (g,p,s) in itertools.product(range(Ng), range(Np), range(Ns))]
print(f"selection = {lst}")
print(f"sum of the values = {-best.energy}")

selection = [[88, 51, 4, 33, 95], [78, 127, 12, 65, 8], [120, 100, 111, 52, 63], [50, 83, 120, 107, 39], [88, 70, 114, 74, 5], [33, 60, 117, 10, 2], [81, 111, 24, 28, 52], [47, 15, 3, 115, 69], [71, 88, 73, 7, 98], [28, 122, 46, 78, 82], [12, 17, 115, 63, 59], [103, 0, 38, 36, 88], [113, 121, 53, 125, 60], [59, 6, 79, 120, 65], [46, 39, 80, 87, 105], [7, 8, 3, 16, 94]]
sum of the values = 15664.0


In [37]:
import numpy as np
np.transpose(np.array(lst))

array([[ 88,  78, 120,  50,  88,  33,  81,  47,  71,  28,  12, 103, 113,
         59,  46,   7],
       [ 51, 127, 100,  83,  70,  60, 111,  15,  88, 122,  17,   0, 121,
          6,  39,   8],
       [  4,  12, 111, 120, 114, 117,  24,   3,  73,  46, 115,  38,  53,
         79,  80,   3],
       [ 33,  65,  52, 107,  74,  10,  28, 115,   7,  78,  63,  36, 125,
        120,  87,  16],
       [ 95,   8,  63,  39,   5,   2,  52,  69,  98,  82,  59,  88,  60,
         65, 105,  94]])

In [184]:
model = pq.Constraint(Hc1[0], "g1").compile()

In [199]:
slots[1,1,1,1,3:]

Array([Binary('slots[1][1][1][1][3]'), Binary('slots[1][1][1][1][4]'), Binary('slots[1][1][1][1][5]'), Binary('slots[1][1][1][1][6]')])

In [186]:
len(model.variables)

280

In [49]:
from pyqubo import Binary, Constraint, Placeholder, Array, OneHotEncInteger, LogEncInteger
weights = [1, 3, 7, 9]
values = [10, 2, 3, 6]
max_weight = 16
# create the array of 0-1 binary variables
# representing the selection of the items
n=len(values)
items = Array.create('item', shape=n, vartype="BINARY")
# define the sum of weights and values using variables
knapsack_weight = sum(
weights[i] * items[i] for i in range(n))
knapsack_value = sum(
values[i] * items[i] for i in range(n))
# define the coefficients of the penalty terms,
# lmd1 and lmd2, using Placeholder class
# so that we can change their values after compilation
lmd1 = Placeholder("lmd1")
lmd2 = Placeholder("lmd2")
# create Hamiltonian and model
weight_one_hot = LogEncInteger("weight_one_hot", value_range=(1, max_weight))
Ha = Constraint((weight_one_hot - knapsack_weight)**2, "weight_constraint")
Hb = knapsack_value
H = lmd2*Ha - Hb
model = H.compile()

In [50]:
feed_dict = {'lmd1': 1, "lmd2": 1}
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [69]:
logenc = LogEncInteger("logenc", value_range=(1, 16))
tstH = -(1 - logenc[3])*logenc[2]*(1-logenc[1])*logenc[0]

TypeError: 'LogEncInteger' object is not subscriptable

In [172]:
logenc = Array.create('logenc', shape=4, vartype="BINARY")
logenc2 = Array.create('logenc2', shape=4, vartype="BINARY")
tstH = -(1 - logenc[3])*logenc[2]*(1-logenc[1])*logenc[0]
tstH = pq.Constraint(sum((logenc[i] - logenc2[i])**2 for i in range(4)), "c1") - delta(logenc, 0b1001, 4)
model = tstH.compile()

In [173]:
delta(logenc, 0b111, 4)

((((1.000000 * Binary('logenc[0]')) * Binary('logenc[1]')) * Binary('logenc[2]')) * (1.000000 + (-1.000000 * Binary('logenc[3]'))))

In [174]:
len(model.variables)
model.variables

['logenc[3]',
 'logenc2[3]',
 'logenc[2]',
 'logenc2[2]',
 'logenc[1]',
 'logenc2[1]',
 'logenc[0]',
 'logenc2[0]',
 'logenc[3] * logenc[0]',
 'logenc[2] * logenc[1]']

In [175]:
delta(logenc, logenc2, 4) 

((((1.000000 * (((2.000000 * Binary('logenc[0]')) * Binary('logenc2[0]')) + (-1.000000 * Binary('logenc2[0]')) + 1.000000 + (-1.000000 * Binary('logenc[0]')))) * (((2.000000 * Binary('logenc[1]')) * Binary('logenc2[1]')) + (-1.000000 * Binary('logenc2[1]')) + 1.000000 + (-1.000000 * Binary('logenc[1]')))) * (((2.000000 * Binary('logenc[2]')) * Binary('logenc2[2]')) + (-1.000000 * Binary('logenc2[2]')) + 1.000000 + (-1.000000 * Binary('logenc[2]')))) * (((2.000000 * Binary('logenc[3]')) * Binary('logenc2[3]')) + (-1.000000 * Binary('logenc2[3]')) + 1.000000 + (-1.000000 * Binary('logenc[3]'))))

In [176]:
qubo, offset = model.to_qubo(feed_dict=feed_dict)
bqm = model.to_bqm(feed_dict=feed_dict)
bqm.normalize()
sampleset = sampler.sample(bqm, num_reads=10, sweeps=1000, beta_range=(1.0, 50.0))
dec_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
best = min(dec_samples, key=lambda x: x.energy)

In [177]:
delta((1,1,1,1), (1,1,1,1), 4)

1

In [182]:
best.constraints()

{'c1': (True, 0.0)}

In [179]:
print(f"selection = {[best.sample[f'logenc[{i}]'] for i in range(4)]}")
print(f"selection = {[best.sample[f'logenc2[{i}]'] for i in range(4)]}")
print(f"sum of the values = {-best.energy}")

selection = [1, 0, 0, 1]
selection = [1, 0, 0, 1]
sum of the values = 1.0


In [79]:
dec_samples

[DecodedSolution({logenc[2] * logenc[0]:1, logenc[0]:1, logenc[1]:0, logenc[2]:1, logenc[3]:0, logenc[3] * logenc[1]:0}, energy=-1.000000),
 DecodedSolution({logenc[0]:1, logenc[2] * logenc[0]:1, logenc[1]:0, logenc[2]:1, logenc[3]:0, logenc[3] * logenc[1]:0}, energy=-1.000000),
 DecodedSolution({logenc[2] * logenc[0]:0, logenc[0]:0, logenc[1]:1, logenc[2]:0, logenc[3]:0, logenc[3] * logenc[1]:0}, energy=0.000000),
 DecodedSolution({logenc[0]:1, logenc[2] * logenc[0]:0, logenc[1]:1, logenc[2]:0, logenc[3]:0, logenc[3] * logenc[1]:0}, energy=0.000000),
 DecodedSolution({logenc[2] * logenc[0]:0, logenc[0]:0, logenc[1]:0, logenc[2]:0, logenc[3]:0, logenc[3] * logenc[1]:0}, energy=0.000000),
 DecodedSolution({logenc[2] * logenc[0]:0, logenc[0]:0, logenc[1]:0, logenc[2]:1, logenc[3]:1, logenc[3] * logenc[1]:0}, energy=0.000000),
 DecodedSolution({logenc[0]:0, logenc[2] * logenc[0]:0, logenc[1]:1, logenc[2]:1, logenc[3]:1, logenc[3] * logenc[1]:1}, energy=0.000000),
 DecodedSolution({logenc[

In [73]:
qubo

{('logenc[3]', 'logenc[1]'): 5.0,
 ('logenc[2] * logenc[0]', 'logenc[2] * logenc[0]'): 14.0,
 ('logenc[2]', 'logenc[0]'): 5.0,
 ('logenc[2]', 'logenc[2] * logenc[0]'): -10.0,
 ('logenc[3]', 'logenc[2] * logenc[0]'): 1.0,
 ('logenc[3]', 'logenc[3] * logenc[1]'): -10.0,
 ('logenc[1]', 'logenc[2] * logenc[0]'): 1.0,
 ('logenc[0]', 'logenc[2] * logenc[0]'): -10.0,
 ('logenc[3] * logenc[1]', 'logenc[3] * logenc[1]'): 15.0,
 ('logenc[2] * logenc[0]', 'logenc[3] * logenc[1]'): -1.0,
 ('logenc[1]', 'logenc[3] * logenc[1]'): -10.0}

In [75]:
tstH = Constraint((weight_one_hot)**3, "weight_constraint")

In [65]:
model2 = (1-)

In [66]:
feed_dict = {'lmd1': 1, "lmd2": 1}
qubo, offset = model2.to_qubo(feed_dict=feed_dict)

In [67]:
qubo

{('weight_one_hot[2]', 'weight_one_hot[0]'): 84.0,
 ('weight_one_hot[1]', 'weight_one_hot[1]'): 26.0,
 ('weight_one_hot[3] * weight_one_hot[2]',
  'weight_one_hot[3] * weight_one_hot[2]'): 1359.0,
 ('weight_one_hot[3]', 'weight_one_hot[0]'): 264.0,
 ('weight_one_hot[3]', 'weight_one_hot[1] * weight_one_hot[0]'): 96.0,
 ('weight_one_hot[3]', 'weight_one_hot[1]'): 576.0,
 ('weight_one_hot[2]', 'weight_one_hot[1]'): 192.0,
 ('weight_one_hot[0]', 'weight_one_hot[3] * weight_one_hot[2]'): 192.0,
 ('weight_one_hot[0]', 'weight_one_hot[1] * weight_one_hot[0]'): -10.0,
 ('weight_one_hot[2]', 'weight_one_hot[1] * weight_one_hot[0]'): 48.0,
 ('weight_one_hot[3]', 'weight_one_hot[2]'): 5.0,
 ('weight_one_hot[1]', 'weight_one_hot[1] * weight_one_hot[0]'): -10.0,
 ('weight_one_hot[2]', 'weight_one_hot[2]'): 124.0,
 ('weight_one_hot[2]', 'weight_one_hot[3] * weight_one_hot[2]'): -10.0,
 ('weight_one_hot[1]', 'weight_one_hot[3] * weight_one_hot[2]'): 384.0,
 ('weight_one_hot[1]', 'weight_one_hot[0]')

In [51]:
weight_one_hot.array

Array([Binary('weight_one_hot[0]'), Binary('weight_one_hot[1]'), Binary('weight_one_hot[2]'), Binary('weight_one_hot[3]')])

In [32]:
qubo

{('weight_one_hot[2]', 'weight_one_hot[1]'): 16.0,
 ('item[3]', 'item[3]'): 57.0,
 ('weight_one_hot[2]', 'item[1]'): -24.0,
 ('weight_one_hot[2]', 'item[0]'): -8.0,
 ('weight_one_hot[3]', 'item[2]'): -112.0,
 ('weight_one_hot[0]', 'item[3]'): -18.0,
 ('weight_one_hot[3]', 'weight_one_hot[0]'): 16.0,
 ('weight_one_hot[3]', 'weight_one_hot[1]'): 32.0,
 ('weight_one_hot[0]', 'item[0]'): -2.0,
 ('weight_one_hot[2]', 'item[2]'): -56.0,
 ('item[0]', 'item[0]'): -11.0,
 ('item[3]', 'item[2]'): 126.0,
 ('weight_one_hot[3]', 'item[0]'): -16.0,
 ('weight_one_hot[1]', 'item[2]'): -28.0,
 ('weight_one_hot[3]', 'weight_one_hot[3]'): 80.0,
 ('weight_one_hot[2]', 'weight_one_hot[2]'): 24.0,
 ('weight_one_hot[3]', 'item[1]'): -48.0,
 ('weight_one_hot[1]', 'item[0]'): -4.0,
 ('weight_one_hot[2]', 'item[3]'): -72.0,
 ('item[1]', 'item[1]'): 1.0,
 ('weight_one_hot[0]', 'item[1]'): -6.0,
 ('item[3]', 'item[0]'): 18.0,
 ('weight_one_hot[1]', 'item[3]'): -36.0,
 ('item[2]', 'item[2]'): 32.0,
 ('weight_one_h

In [33]:
pip install dwave-neal

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import neal
# use simulated annealing (SA) sampler of neal package
sampler = neal.SimulatedAnnealingSampler()
feasible_sols = []
# search the best parameters: lmd1 and lmd2
for lmd1_value in range(1, 10):
    for lmd2_value in range(1, 10):
        feed_dict = {'lmd1': lmd1_value, "lmd2": lmd2_value}
        qubo, offset = model.to_qubo(feed_dict=feed_dict)
        bqm = model.to_bqm(feed_dict=feed_dict)
        bqm.normalize()
        sampleset = sampler.sample(bqm, num_reads=10, sweeps=1000, beta_range=(1.0, 50.0))
        dec_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
        best = min(dec_samples, key=lambda x: x.energy)
        # store the feasible solution
        if not best.constraints(only_broken=True):
            feasible_sols.append(best)
best_feasible = min(feasible_sols, key=lambda x: x.energy)
print(f"selection = {[best_feasible.sample[f'item[{i}]'] for i in range(n)]}")
print(f"sum of the values = {-best_feasible.energy}")

selection = [1, 1, 0, 1]
sum of the values = 18.0


In [21]:
feasible_sols

[DecodedSolution({weight_one_hot[0]:0, item[0]:1, weight_one_hot[1]:0, item[1]:0, weight_one_hot[2]:0, item[2]:0, weight_one_hot[3]:0, item[3]:1, weight_one_hot[4]:0, weight_one_hot[5]:0, weight_one_hot[6]:0, weight_one_hot[7]:0, weight_one_hot[8]:0, weight_one_hot[9]:1}, energy=-16.000000),
 DecodedSolution({weight_one_hot[0]:0, item[0]:1, weight_one_hot[1]:0, item[1]:0, weight_one_hot[2]:0, item[2]:0, weight_one_hot[3]:0, item[3]:1, weight_one_hot[4]:0, weight_one_hot[5]:0, weight_one_hot[6]:0, weight_one_hot[7]:0, weight_one_hot[8]:0, weight_one_hot[9]:1}, energy=-16.000000),
 DecodedSolution({weight_one_hot[0]:0, item[0]:1, weight_one_hot[1]:0, item[1]:0, weight_one_hot[2]:0, item[2]:0, weight_one_hot[3]:0, item[3]:1, weight_one_hot[4]:0, weight_one_hot[5]:0, weight_one_hot[6]:0, weight_one_hot[7]:0, weight_one_hot[8]:0, weight_one_hot[9]:1}, energy=-16.000000),
 DecodedSolution({weight_one_hot[0]:0, item[0]:1, weight_one_hot[1]:0, item[1]:0, weight_one_hot[2]:0, item[2]:0, weight_